In [12]:
%run purpleair/merge-pa.py --csv bigtable.csv --folder purpleair/sample

In [14]:
%run worldpop/load-wp.py --csv bigtable.csv --folder worldpop/GIS

In [15]:
%run cyclic/build-cyclic.py --csv bigtable.csv --column datetime

In [43]:
import pandas as pd

df = pd.read_csv("bigtable.csv")
df['time'] = pd.to_datetime(df["datetime"],format="%Y-%m-%d %H:%M:%S %Z")
df['date'] = pd.Series(df.time).dt.strftime("%Y-%m-%d")
df['hour'] = pd.Series(df.time).dt.strftime("%H")

print("nb ground station measurement : {}".format(len(df)))


#2019-01-01T02:40:00
df_meteo = pd.read_csv("weather/weather.csv")
df_meteo['time'] = pd.to_datetime(df_meteo["DATE"],format="%Y-%m-%dT%H:%M:%S")
df_meteo['date'] = pd.Series(df_meteo.time).dt.strftime("%Y-%m-%d")
df_meteo['hour'] = pd.Series(df_meteo.time).dt.strftime("%H")
df_meteo['datehour'] = pd.Series(df_meteo.time).dt.strftime("%Y-%m-%d-%H")

print("nb meteo : {}".format(len(df_meteo)))

df_meteo.head()

    

nb ground station measurement : 105054
nb meteo : 14449


,DATE,Temperature,Wind,Dew,Sky,Visibility,ATM,time,date,hour,datehour
0,2019-01-01T00:00:00,"+0089,1","999,9,C,0000,1","+0078,1","99999,9,9,N","006000,1,9,9","10165,1",2019-01-01 00:00:00,2019-01-01,00,2019-01-01-00
1,2019-01-01T00:11:00,"+0089,5","999,9,C,0000,5","+0078,5","00152,5,M,N","004023,5,N,5","99999,9",2019-01-01 00:11:00,2019-01-01,00,2019-01-01-00
2,2019-01-01T00:17:00,"+0089,5","999,9,C,0000,5","+0078,5","00122,5,M,N","002414,5,N,5","99999,9",2019-01-01 00:17:00,2019-01-01,00,2019-01-01-00
3,2019-01-01T00:22:00,"+0083,5","999,9,C,0000,5","+0067,5","00122,5,M,N","001207,5,N,5","99999,9",2019-01-01 00:22:00,2019-01-01,00,2019-01-01-00
4,2019-01-01T00:47:00,"+0080,5","999,9,C,0000,5","+0070,5","00122,5,M,N","001609,5,N,5","99999,9",2019-01-01 00:47:00,2019-01-01,00,2019-01-01-00


In [44]:

df_meteo=df_meteo.groupby(['datehour']).first().reset_index()

print("nb meteo unique hourly : {}".format(len(df_meteo)))

df_meteo.head()


nb meteo unique hourly : 8759


,datehour,DATE,Temperature,Wind,Dew,Sky,Visibility,ATM,time,date,hour
0,2019-01-01-00,2019-01-01T00:00:00,"+0089,1","999,9,C,0000,1","+0078,1","99999,9,9,N","006000,1,9,9","10165,1",2019-01-01 00:00:00,2019-01-01,00
1,2019-01-01-01,2019-01-01T01:14:00,"+0089,5","999,9,C,0000,5","+0078,5","00091,5,M,N","003219,5,N,5","99999,9",2019-01-01 01:14:00,2019-01-01,01
2,2019-01-01-02,2019-01-01T02:10:00,"+0094,5","200,5,N,0015,5","+0083,5","00091,5,M,N","003219,5,N,5","99999,9",2019-01-01 02:10:00,2019-01-01,02
3,2019-01-01-03,2019-01-01T03:00:00,"+0100,1","220,1,N,0031,1","+0089,1","99999,9,9,N","004800,1,9,9","10127,1",2019-01-01 03:00:00,2019-01-01,03
4,2019-01-01-04,2019-01-01T04:52:00,"+0106,5","200,5,N,0026,5","+0094,5","00091,5,W,N","001609,5,N,5","10106,5",2019-01-01 04:52:00,2019-01-01,04


In [33]:
moments = df[['date', 'hour']].apply(tuple, axis=1).tolist()
moments = list(set(moments))
print("nb moments : {}".format(len(dates)))


nb moments : 4392


In [ ]:
new_df = pd.merge(left=df, right=df_meteo, how='left', 
                  left_on=['date','hour'], right_on=['date','hour'])

print("nb ground station measurement after : {}".format(len(new_df)))
print("nb ground station measurement before : {}".format(len(df)))


new_df = new_df.drop(["time_x", "hour", "time_y"], axis=1)

new_df.head()
new_df.to_csv("bigtable.csv", index=False)

nb ground station measurement after : 105054
nb ground station measurement before : 105054
